In [2]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd
import numpy as np
import glob

In [33]:
os.chdir('C:/Users/zph11/OneDrive - Washington University in St. Louis/WashU/Courses/S3/Capstone/data/tracab/extracted')

In [3]:
t = pd.read_csv('AUS_STL_tracab.csv')

In [4]:
# Calculate the Euclidean distance
t ['distance'] = np.sqrt((t['176'].diff().fillna(0))**2 + 
                           (t['177'].diff().fillna(0))**2 + 
                           (t['178'].diff().fillna(0))**2)

In [5]:
t

,FrameCount,176,177,178,179,180,181,Second,distance
0,2122262,12,-4,0,1.12,H,Alive,0,0.000000
1,2122263,-64,-9,0,1.08,H,Alive,0,76.164296
2,2122264,-140,-13,0,1.08,H,Alive,0,76.105190
3,2122265,-217,-19,0,1.08,H,Alive,0,77.233412
4,2122266,-292,-24,0,81.68,H,Alive,0,75.166482
...,...,...,...,...,...,...,...,...,...
181435,2330198,166,3674,0,1.91,A,Dead,6911,2.000000
181436,2330199,165,3674,0,1.33,A,Dead,6911,1.000000
181437,2330200,162,3673,0,1.84,A,Dead,6911,3.162278
181438,2330201,160,3673,0,1.91,A,Dead,6911,2.000000


In [6]:
# Group the data by second
grouped = t.groupby('Second')

In [7]:
# Calculate the total distance
total_distance_per_second = grouped['distance'].sum().reset_index()

# Rename the columns for clarity
total_distance_per_second.columns = ['second', 'total_distance']

# Display the result
total_distance_per_second

,second,total_distance
0,0,1360.504006
1,1,101.517905
2,2,281.999158
3,3,374.985258
4,4,1510.386349
...,...,...
6044,6907,324.799831
6045,6908,109.604905
6046,6909,90.535110
6047,6910,57.536631


In [8]:
total_distance_per_second.to_csv('t.csv', index = False)

In [9]:
tracab = glob.glob('*.csv')

In [ ]:
for file in tracab:
    # Load the data from the current file
    data = pd.read_csv(file)
    
    # Calculate the Euclidean distance
    data['Distance'] = np.sqrt((data['176'].diff().fillna(0))**2 + 
                               (data['177'].diff().fillna(0))**2 + 
                               (data['178'].diff().fillna(0))**2)
    
    # Optionally, save the distance data to a new CSV file
    output_file_name = f'distance_{file}'
    data.to_csv(output_file_name, index=False)

In [34]:
velo = glob.glob('*.csv')

velocity = []

for file in velo:
        data = pd.read_csv(file)
        idx_max_velocity = data.groupby('Second')['179'].idxmax()
        max_velocity_rows = data.loc[idx_max_velocity]
        max_velocity_rows.reset_index(drop=True, inplace=True)
        velocity.append(max_velocity_rows)
        

In [35]:
velocity[1]

,FrameCount,176,177,178,179,180,181,Second
0,1310988,-1665,217,8,70.20,H,Alive,0
1,1311012,-2117,265,8,35.00,H,Alive,1
2,1311051,-1720,203,87,98.78,H,Alive,2
3,1311053,-1558,253,148,97.75,H,Alive,3
4,1311083,419,978,675,67.05,H,Alive,4
...,...,...,...,...,...,...,...,...
5888,1515437,-1569,430,116,8.70,A,Dead,6808
5889,1515465,-1527,218,60,12.59,A,Dead,6809
5890,1515502,-1353,261,113,5.65,A,Dead,6810
5891,1515503,-1350,259,113,5.95,A,Dead,6811


In [36]:
# Define the directory where you want to save the files
output_directory = 'velocity/'

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate through the filtered data list and the original file paths together
for filtered_data, file_path in zip(velocity, velo):
    # Create a new file name based on the original file name
    new_file_name = os.path.basename(file_path).replace('_tracab.csv', '.csv')
    
    # Join the new file name with the output directory to get the full path
    new_file_path = os.path.join(output_directory, new_file_name)
    
    # Write the filtered data to the new CSV file
    filtered_data.to_csv(new_file_path, index=False)

In [11]:
os.chdir('C:/Users/zph11/OneDrive - Washington University in St. Louis/WashU/Courses/S3/Capstone/data/tracab/extracted/op')

In [12]:
opta = glob.glob('*.csv')

In [ ]:
# Get a list of all CSV files in the working directory
takeon = glob.glob('*.csv')

for file in takeon:
    # Load the data from the current file
    data = pd.read_csv(file)
    
    # Initialize a new column 'successful' with NaN values
    data['successful'] = None
    
    # Iterate through the rows of the DataFrame
    for i in range(len(data) - 1):  # -1 to avoid index error on the last row
        # Check if the current row is a 'TakeOn' event
        if data.loc[i, 'playType'] == 'TakeOn':
            # Check the 'playType' value in the next row
            if data.loc[i + 1, 'playType'] == 'Tackle':
                # If the next row is a 'Tackle', the 'TakeOn' failed
                data.loc[i, 'successful'] = 0
            else:
                # Otherwise, the 'TakeOn' was successful
                data.loc[i, 'successful'] = 1
    
    # Save the updated DataFrame back to the CSV file
    data.to_csv(file, index=False)

In [13]:
# Initialize an empty list to hold the filtered data
filtered_data_list = []

for file in opta:
    # Load the data from the current file
    data = pd.read_csv(file)
    
    # Select specific columns (replace 'column_label1', 'column_label2', etc. with your actual column labels)
    selected_columns = data[["toucher", "playType", "successful", "Second", "sequenceSeconds", 
                             "sequenceSeconds", "Team.1", "xG", "xA", "ShotDist"]]
    
    # Filter rows based on certain values (replace 'column_label1', 'value1', etc. with your actual column labels and values)
    filtered_rows = selected_columns[(selected_columns['Team.1'] == 'STL') &
                                     (selected_columns['playType'] == 'TakeOn')]
    
    # Append the filtered data to the list
    filtered_data_list.append(filtered_rows)

In [14]:
# Define the directory where you want to save the files
output_directory = 'filtered_data_files/'

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate through the filtered data list and the original file paths together
for filtered_data, file_path in zip(filtered_data_list, opta):
    # Create a new file name based on the original file name
    new_file_name = os.path.basename(file_path).replace('.csv', '_filtered.csv')
    
    # Join the new file name with the output directory to get the full path
    new_file_path = os.path.join(output_directory, new_file_name)
    
    # Write the filtered data to the new CSV file
    filtered_data.to_csv(new_file_path, index = False)

In [ ]:
print(filtered_data_list)

In [42]:
os.chdir('C:/Users/zph11/OneDrive - Washington University in St. Louis/WashU/Courses/S3/Capstone/data/tracab/extracted/velocity')

In [44]:
all_csv_files = glob.glob('*.csv')
group1_files = [file for file in all_csv_files if 'opta' not in file]
group2_files = [file for file in all_csv_files if 'opta' in file]
# Create a dictionary to hold the result DataFrames
result_dfs = {}

# Iterate through the first group of files
for file1 in group1_files:
    # Extract the match name from the file name
    match_name = file1.replace('.csv', '')
    
    # Find the corresponding file in the second group
    corresponding_file2 = f'{match_name}_opta_filtered.csv'
     # Check if the corresponding file exists in the second group
    if corresponding_file2 in group2_files:
        # Read the CSV files into Pandas DataFrames
        df1 = pd.read_csv(file1)
        df2 = pd.read_csv(corresponding_file2)
        
        # Perform an inner join on the desired columns (adjust the columns as needed)
        result_df = pd.merge(df1, df2, on='Second', suffixes=('_group1', '_group2'))
        
        # Add the result DataFrame to the dictionary
        result_dfs[match_name] = result_df
        
        # Optionally, write the result to a new CSV file
        result_file_path = os.path.join('results', f'{match_name}_joined.csv')
        os.makedirs(os.path.dirname(result_file_path), exist_ok=True)
        result_df.to_csv(result_file_path, index=False)

# Now result_dfs contains the joined DataFrames for each match

In [45]:
all_csv_files

['AUS_STL.csv',
 'AUS_STL_opta_filtered.csv',
 'CHI_STL.csv',
 'CHI_STL_opta_filtered.csv',
 'COL_STL.csv',
 'COL_STL_opta_filtered.csv',
 'DAL_STL.csv',
 'DAL_STL_opta_filtered.csv',
 'LAFC_STL.csv',
 'LAFC_STL_opta_filtered.csv',
 'NSH_STL.csv',
 'NSH_STL_opta_filtered.csv',
 'POR_STL.csv',
 'POR_STL_opta_filtered.csv',
 'RSL_STL.csv',
 'RSL_STL_opta_filtered.csv',
 'SEA_STL.csv',
 'SEA_STL_opta_filtered.csv',
 'SJ_STL.csv',
 'SJ_STL_opta_filtered.csv',
 'STL_CIN.csv',
 'STL_CIN_opta_filtered.csv',
 'STL_CLT.csv',
 'STL_CLT_opta_filtered.csv',
 'STL_COL.csv',
 'STL_COL_opta_filtered.csv',
 'STL_HOU.csv',
 'STL_HOU_opta_filtered.csv',
 'STL_LA.csv',
 'STL_LA_opta_filtered.csv',
 'STL_MIA.csv',
 'STL_MIA_opta_filtered.csv',
 'STL_MIN.csv',
 'STL_MIN_opta_filtered.csv',
 'STL_POR.csv',
 'STL_POR_opta_filtered.csv',
 'STL_RSL.csv',
 'STL_RSL_opta_filtered.csv',
 'STL_SJ.csv',
 'STL_SJ_opta_filtered.csv',
 'STL_SKC.csv',
 'STL_SKC_opta_filtered.csv',
 'STL_VAN.csv',
 'STL_VAN_opta_filtere

In [4]:
os.chdir('C:/Users/zph11/OneDrive - Washington University in St. Louis/WashU/Courses/S3/Capstone/data/tracab/extracted/velocity/joined_data')

In [5]:
goal = pd.read_excel('combined_goal_assist_data.xlsx')
takeon = pd.read_excel('combined_takeon_data.xlsx')

In [6]:
merged_df = pd.merge(goal, takeon, on = ['Team.1','match'])
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("merged_file.xlsx", engine = 'xlsxwriter')
# Write the merged DataFrame to the new excel file
merged_df.to_excel(writer, sheet_name = 'Sheet1', index = False)

# Save the result
writer.save()

C:\Users\zph11\AppData\Local\Temp\ipykernel_676\1770296439.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
